## Exploration Data Analysis (EDA)

* Что такое исследовательский анализ данных? 
* Как убедиться, что данные готовы для применения алгоритмов машинного обучения в проекте?
* Как выбрать наиболее подходящее семейство алгоритмов для набора данных?
* Как определить переменные, которые могут быть использованы для обучения модели?

Это часть вопросов, на которые можно ответить с помощью исследовательского анализа данных. Это подход для обобщения, визуализации и детального ознакомления с характеристиками набора данных. Предварительный анализ данных полезен для проектов, которыми мы занимаемся, посколько он позволяет быть уверенным в том, что будущие результаты моделирования, будут устойчивыми, правильно интерпретируемыми и применимыми к текущему бизнес-процессу, в рамках которого решается задача. Такой уровень уверенности обеспечивается за счет того, что сырые данные будут проверены на наличие аномалий и ошибочных значений, что позволит убедится в корректности собранных данных. EDA также помогает найти идеи, которые были не очевидны при первичном (беглом) анализе данных.

Кроме того, EDA выполняется для отбора переменных, которые будут использоваться для обучения моделей машинного обучения. Как правило в соревнованиях по машинному обучению, мы все время возвращаемся к этапу разработки новых признаков, посколько первоначальные признаки не позволят занять высокие места в соревновании. После окончания предварительного анализа данных, мы получаем первоначальный набор признаков, которые могут быть использованы для построения baseline-решения.

## Методы и инструменты EDA

* Одномерная визуализация - использование сводной статистики для каждого поля в наборе сырых данных.

* Двумерная визуализация - выполняется для определения взаимосвязи между каждой переменной в наборе данных и интересующей (целевой) переменной.

* Многомерная визуализация - выполняется для понимания взаимодействия между различными полями в наборе данных.

* Снижение размерности - помогает понять какие признаки в наборе данных объясняют наибольшую разницу между наблюдениями, и позволят снизить размерность задачи, сохранив большую часть полезного сигнала.

* Анализ количества пропусков и возможных способов их устранения.

С помощью этих методов Data Scientist'ы проверяют гипотезы и выявляют закономерности, которые позволяют понять бизнес-задачу и выбрать семейство моделей, которые подойдут для решения задачи.

## Почему пропустить EDA - плохая идея?

Всегда можно пропустить этап Exploration Data Analysis и перейти сразу к этапу построения моделей, или сделать поверхностный, это часто встречается у новичков в Data Science. Такое небрежное отношение может привести к искажению данных из-за наличия выбросов или большом количестве пропущенных значений в признаке, и следовательно, к печальным результатам для проекта, в целом. 

Последствия:

* создание некачественных моделей;
* создание хороших моделей на искаженных данных;
* выбор неправильных признаков для передачи в модель;
* неэффективное использование ресурсов из-за использования некачественных признаков;

EDA используется, с одной стороны, для ответа на вопросы, проверки бизнес-предположений, создания гипотез и дальнейшего анализа. С другой стороны, мы можем использовать этап EDA для подготовки данных к моделирования. 

## Ключевые концепции Exploration Data Analysis

__2 типа анализа данных__
* подкрепляющий анализ данных
* исследовательский анализ данных

__4 цели EDA__
* выявить основные паттерны в данных
* выявить аномалии в данных
* сформировать гипотезы о дальнейших способах создания признаков
* проверить бизнесовые предположения

__2 метода исследования__
* одномерный анализ данных
* двумерный анализ данных

__Основные составляющие качественного EDA__
* тренды в данных
* распределения
* статистики: среднее, медиана, мода
* наличие выбросов
* корреляция
* визуальное исследование

In [10]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp, shapiro, ttest_ind, mannwhitneyu, chi2_contingency
from statsmodels.stats.weightstats import zconfint

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, ShuffleSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno
import xgboost as xgb
%matplotlib inline

# убираем лишние замечания интерпретатора
import warnings
warnings.simplefilter('ignore')

# import additional packages for WOE/IV
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
# глобальные переменные WOE/IV
max_bin = 20
force_bin = 3

pd.options.display.max_columns = 50

In [2]:
# функции метода WOE/IV

def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


X_train = train.copy()
X_train['ID'] = X_train.index.tolist()
#X_train = X_train.drop('ID_code', axis = 1)
X_test = test.copy()
X_test['ID'] = X_test.index.tolist()
#X_test = X_test.drop('ID_code', axis = 1)

print("X_train.shape = {} rows, {} cols".format(*X_train.shape))
print("X_test.shape = {} rows, {} cols".format(*X_test.shape))

X_train.shape = 10000 rows, 21 cols
X_test.shape = 5000 rows, 20 cols


In [ ]:
X_train.describe()

Предварительный вывод: распределения в трейне и тесте очень похожи, это хорошо

In [7]:
X_train.head()

Id  DistrictId  Rooms     Square  LifeSquare  KitchenSquare  Floor  \
0  14038          35    2.0  47.981561   29.442751            6.0      7   
1  15053          41    3.0  65.683640   40.049543            8.0      7   
2   4765          53    2.0  44.947953   29.197612            0.0      8   
3   5809          58    2.0  53.352981   52.731512            9.0      8   
4  10783          99    1.0  39.649192   23.776169            7.0     11   

   HouseFloor  HouseYear  Ecology_1  ... Ecology_3 Social_1  Social_2  \
0         9.0       1969   0.089040  ...         B       33      7976   
1         9.0       1978   0.000070  ...         B       46     10309   
2        12.0       1968   0.049637  ...         B       34      7759   
3        17.0       1977   0.437885  ...         B       23      5735   
4        12.0       1976   0.012339  ...         B       35      5776   

   Social_3  Healthcare_1  Helthcare_2  Shops_1  Shops_2          Price  ID  
0         5           NaN            0       11        B  184966.930730   0  
1         1         240.0            1       16        B  300009.450063   1  
2         0         229.0            1        3        B  220925.908524   2  
3         3        1084.0            0        5        B  175616.227217   3  
4         1        2078.0            2        4        B  150226.531644   4  

[5 rows x 21 columns]

## Анализ числовых переменных

In [14]:
categorical_features = X_train.select_dtypes(include=['object'])
print(f"count of categorical_features {categorical_features.shape[1]}")

cat_features = categorical_features.columns.to_list()
X_train[cat_features].head()

count of categorical_features 3


Ecology_2 Ecology_3 Shops_2
0         B         B       B
1         B         B       B
2         B         B       B
3         B         B       B
4         B         B       B

In [8]:
numerical_features = X_train.select_dtypes(include=[np.number])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features.columns

count of numeric_features 18


Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price', 'ID'],
      dtype='object')

In [11]:
numerical_features.head()

Id  DistrictId  Rooms     Square  LifeSquare  KitchenSquare  Floor  \
0  14038          35    2.0  47.981561   29.442751            6.0      7   
1  15053          41    3.0  65.683640   40.049543            8.0      7   
2   4765          53    2.0  44.947953   29.197612            0.0      8   
3   5809          58    2.0  53.352981   52.731512            9.0      8   
4  10783          99    1.0  39.649192   23.776169            7.0     11   

   HouseFloor  HouseYear  Ecology_1  Social_1  Social_2  Social_3  \
0         9.0       1969   0.089040        33      7976         5   
1         9.0       1978   0.000070        46     10309         1   
2        12.0       1968   0.049637        34      7759         0   
3        17.0       1977   0.437885        23      5735         3   
4        12.0       1976   0.012339        35      5776         1   

   Healthcare_1  Helthcare_2  Shops_1          Price  ID  
0           NaN            0       11  184966.930730   0  
1         240.0            1       16  300009.450063   1  
2         229.0            1        3  220925.908524   2  
3        1084.0            0        5  175616.227217   3  
4        2078.0            2        4  150226.531644   4

вывод - 18 фич - цифровые

###### Выявление дискретных признаков

In [15]:
discrete_feature = [
    feature for feature in numerical_features
    if len(X_train[feature].unique())<25 and feature not in ["ID"]
]

print(f"Discrete Variables Count: {discrete_feature}")

Discrete Variables Count: ['Rooms', 'Helthcare_2', 'Shops_1']


Вывод: у нас 3 дискретные фичи

###### Непрерывные признаки

In [ ]:
continuous_feature = [
    feature for feature in numerical_features
    if feature not in discrete_feature and feature not in ["ID"]
]

print(f"Continuous Feature Count {len(continuous_feature)}")

Все числовые признаки - непрерывные, ID исключили

Для непрерывные признаков имеет смысл строить гистрограммы распределения или функции распределения, а также зависимость непрерывного признака в разрезе целевой переменной. Кроме того, имеет смысл строить распределение __одновременно__ и для обучающей выборки, и для тестовой для понимания насколько отличаются распределения.

In [ ]:
# гистограммы распределения признаков:

fig, axes = plt.subplots(50, 4, figsize=(15, 150))

for num, feature in enumerate(continuous_feature):
    data = train[feature].copy()
    axes[num//4, num%4].hist(data, bins=50, color="blue", ec='white')
    axes[num//4, num%4].set_xlabel(feature, fontsize=10)

Вывод: некоторые распределения нормальны, некоторые нет, имеют скосы, также, у некоторых имеются среднего размера
    выбросы свидетельствующие или о предобработке признаков (работа с пропусками) или какой-либо полезной информации для нас.
    Это наблюдается у следующих признаков: 2, 9, 12, 13, 16, 37, 40, 41, 53, 80, 81, 101, 108, 126, 134, 153, 158, 175,
        188, 196. Запомним их, если их важность подтвердиться, то можно проверить эти выбросы в разрезе целевой переменной,
        есть ли связь, как это делается для пропусков

In [ ]:
# Теперь посмотрим визуально на распределение в трейне и тесте, есть ли различия

fig, axes = plt.subplots(50, 4, figsize=(20, 200))

for num, feature in enumerate(continuous_feature):
    try:
        train_data, test_data = train[feature].copy(), test[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//4, num%4], color="blue", label="train")
        sns.kdeplot(test_data, ax=axes[num//4, num%4], color="green", label="test")
    except RuntimeError:
        pass
    except KeyError:
        train_data = train[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//4, num%4], color="blue", label="train")
    axes[num//4, num%4].set_title(feature, fontsize=10)

Вывод: train and test из одного датасета, распределения близки, соответсвенно высока вероятность стабильности будущей модели

## Анализ пропусков

In [ ]:
msno.matrix(X_train)

нет пропусков

###### Анализ "скошенности" распределений:

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_stats, test_stats = X_train.skew(), X_test.skew()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.xlim(0, len(train_stats))
plt.ylabel("skew", size=15)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_stats, test_stats = X_train.kurtosis(), X_test.kurtosis()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.ylabel("kurtosis", size=15)
plt.xlim(0, len(train_stats))
plt.show()

Вывод: "скошенность" одинакова на трейне и тесте - можно использовать все фичи

## Анализ распределения целевой переменной

###### Проверка баланса классов цели¶

In [ ]:
X_train['target'].value_counts()

In [ ]:
plt.figure(figsize=(4, 2))

sns.countplot(x='target', data=X_train)

plt.title('Target variable distribution')
plt.show()

Вывод: сильный дисбаланс, надо делать балансировку

### Для дальнейшего изучения взаимосвязи признаков будем использовать:
* Correlation Heat Map
* Heat Map для отдельных признаков
* Pair Plot
* Scatter Plot

## Корреляция между числовыми признаками

In [ ]:
correlation = numerical_features.corr()
corr_with_target = correlation["target"].sort_values(ascending = False)
corr_with_target

на трейне линейной корреляции между фичами и целью не видно

In [ ]:
# Тепловая карта - лучший способ __быстро__ получить информацию о силе связи между переменными.

fig, axes = plt.subplots(figsize = (50,50))
plt.title("Correlation of Numeric Features with Sale Price", y=1, size=16)
sns.heatmap(correlation, square=True, vmax=0.8, cmap="viridis")

In [ ]:
# 20 наиболее коррелированных фич

k = 20
cols = correlation.nlargest(k, "target")["target"].index
cm = np.corrcoef(train[cols].values.T)
fig, axes = plt.subplots(figsize = (30, 12))

sns.heatmap(
    cm,
    vmax=.8,
    linewidths=0.01,
    cmap="viridis",
    square=True,
    annot=True,
    linecolor="white",
    xticklabels=cols.values,
    annot_kws = {"size": 9},
    yticklabels=cols.values
)

###### Вывод: нет корреляции

теперь посмотрим в сравнении трейн и тест:

In [ ]:
train_correlations = numerical_features.corr()
train_correlations = train_correlations.values.flatten()
train_correlations = train_correlations[train_correlations != 1]

columns = numerical_features.columns.tolist()
columns.remove("target")

test_correlations = X_test[columns].corr()
test_correlations = test_correlations.values.flatten()
test_correlations = test_correlations[test_correlations != 1]

plt.figure(figsize=(20,5))
sns.distplot(train_correlations, color="Blue", label="train")
sns.distplot(test_correlations, color="Green", label="test")

plt.xlabel("Correlation values found in train (except 1)", size=14)
plt.title("Are there correlations between features?", size=14)
plt.legend(loc="best", fontsize=14)
plt.ylabel("Density", size=14)

трейн и тест одинаково показывают линейную корреляцию фичей к цели -> трейн и тест из одного сета,
модель точно будет стабильна в логистической регрессии

## Нелинейная связь между признаками

Коэффициент корреляции позволяет установить линейную силу связи между признаками, но также признаки могут быть связаны нелинейно, что сложно определяется коэффициентом корреляции: может быть ситуация, что признаки очень сильно связаны между собой, но коэффициент корреляции равен 0. 

Для начала давайте воспользуемся случайным лесом, чтобы выбрать 10 наиболее значимых признаков. Они могут служить отправной точкой для понимания природы данных и общего понимания данные. Кроме того, они могут дать некоторые идеи о том, как создавать новые признаки. Давайте начнем!

In [ ]:
target = train["target"]

parameters = {"max_depth": 6, "n_estimators": 25, "random_state": 27, "n_jobs": 2}

forest = RandomForestRegressor(**parameters)
forest.fit(numerical_features.drop("target", axis=1), target)

In [ ]:
n_top = 30
importances = forest.feature_importances_
idx = np.argsort(importances)[::-1][0:n_top]
feature_names = numerical_features.drop("target", axis=1).columns

plt.figure(figsize=(30, 5))
sns.barplot(x=feature_names[idx], y=importances[idx], palette="viridis")
plt.title("What are the top important features to start with?", size=14)

Итак, получаем 20 наиболее (нелинейно) влиятельных фич: 81, 12, 139, 26, 53, 110, 174, 109, 22, 146, 166, 80, 6,
165, 0, 78, 13, 179, 148, 108.

Ранее мы отметили следующие фичи: 2, 9, 12, 13, 16, 37, 40, 41, 53, 80, 81, 101, 108, 126, 134, 153, 158, 175, 188, 196

Видим, что 6 фич: 12, 13, 53, 80, 81, 108 сильно влияют на цель, при этом имеют выбросы, соответственно имеет 
смысл сделать на основе выбросов новые фичи - например, в выбросе - "1", в остальных местах "0"

## Pair Plot

PairPlor между `target` и высоко-коррелированными переменными. Строим этот график для лучшего понимания характерика связи между целевой переменной и признаком. С помощью корреляционной матрицы мы установили что связь между определенными признаками и целевой переменной есть, но характер этой связи пока остается невыясненным. Строим PairPlot для лучшего понимания характера этой связи;

In [ ]:
sns.set()
columns = [
    "target", "var_0", "var_6", "var_12", "var_13", "var_22",
    "var_26", "var_53", "var_78", "var_80", "var_81", "var_108", "var_109", "var_110",
    "var_139", "var_146", "var_148", "var_165", "var_166", "var_174", "var_179"
]

sns.pairplot(X_train[columns], size=2, kind="scatter", diag_kind="kde")
plt.show()


__Короткий анализ:__

* явных зависимостей нет, но есть границы в некоторых зависимостях за которые признаки почти не уходят, отмечаем
эти признаки

## Scatter Plot

Scatter-plot'ы между наиболее скореллированными с целью признаками

In [ ]:
features = [
    "var_0", "var_6", "var_12", "var_13", "var_22",
    "var_26", "var_53", "var_78", "var_80", "var_81", "var_108", "var_109", "var_110",
    "var_139", "var_146", "var_148", "var_165", "var_166", "var_174", "var_179" 
]

fig, axes = plt.subplots(5, 4, figsize=(15, 30))
for num, feature in enumerate(features):
    sns.regplot(
        x=feature,
        y="target",
        data=X_train,
        ax=axes[num%5, num//5],
        scatter=True,
        fit_reg=True
    )

вывод - везде видны зависимости от величин признаков, что подтверждает их правильный выбор

###### Теперь оценим фичи по методу WOE/IV

In [ ]:
final_iv, IV = data_vars(X_train, X_train['target']) # вычисление уровней IV (Information Values) метода WOE по фичам

In [ ]:
IV.sort_values('IV') # вывод IV данных по фичам в отсортированном виде

Согласно критерию 0.02, оставляем 30 фич с показателем более 0.02, а именно:
    139, 6, 174, 76, 146, 110, 12, 26, 21, 148, 34, 22, 115, 165, 80, 133, 170, 53, 166, 81, 149, 99, 13, 190, 67,
    169, 0, 78, 198, 192
До этого мы получили 20 наиболее (нелинейно) влиятельных фич:
    81, 12, 139, 26, 53, 110, 174, 109, 22, 146, 166, 80, 6, 165, 0, 78, 13, 179, 148, 108
Видим, что кроме 109, 179 и 108 признака, остальные совпадают. Оставляем их в пуле

В самом начале мы отметили фичи: 2, 9, 12, 13, 16, 37, 40, 41, 53, 80, 81, 101, 108, 126, 134, 153, 158, 175,
        188, 196, из них можно оставить 108-ую

###### Теперь проверим распределения выбранных фич в зависимости от целевой функции "0" или "1" 

In [ ]:
features = [
    "var_0", "var_6", "var_12", "var_13", "var_22",
    "var_26", "var_53", "var_78", "var_80", "var_81", "var_108", "var_110",
    "var_139", "var_146", "var_148", "var_165", "var_166", "var_174"
]

fig, axes = plt.subplots(6, 3, figsize=(15, 30))
for num, feature in enumerate(features):
    try:
        feature_with_target_s = X_train[[feature, 'target']].sample(1000)
        feature_s = feature_with_target_s[feature]
        feature_target_0 = feature_s[feature_with_target_s['target'] == 0]
        feature_target_1 = feature_s[feature_with_target_s['target'] == 1]
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    axes[num//3, num%3].set_title(feature, fontsize=10)   

Вывод: у всех у них заметна разница в распределениях для целей 0 и 1, правильные фичи оставляем

## Анализ распределения на обучении и тесте

Совпадение распределений на выборке для обучения и для тестирования - одна из ключевых коцепций статистического обучения. Обучая модель на некоторых данных, мы принимаем гипотезу о том, что модель должна работать на таких же данных. Хотя ранее мы уже косвенно убедились в хороших данных, но всё же проверим ещё раз

## Kolmogorov-Smirnov Test

In [ ]:
statistics_array = []
fig, axes = plt.subplots(15, 2, figsize=(20, 80))

for num, feature in enumerate(feature_names[idx]):
    statistic, pvalue = ks_2samp(
        X_train[feature].sample(500), X_test[feature].sample(500)
    )
    statistics_array.append(statistic)

    sns.kdeplot(X_train[feature], ax=axes[num//2, num%2], color="Blue", label="Train")
    sns.kdeplot(X_test[feature], ax=axes[num//2, num%2], color="Green", label="Test")

    axes[num//2, num%2].set_title(
        (f"Kolmogorov-Smirnov test for train/test\n"
        f"feature: {feature}, p-value: {round(statistic, 3)}")
    )

На основе графиков и величин pvalue, видно, что распределения в трейне и тесте схожи

## Population Stability Index

In [ ]:
class PSI(BaseEstimator, TransformerMixin):
    """
    Вычисление PSI и отбор признаков на их основе.

    Parameters
    ----------
    threshold: float
        Порог для отбора переменных по PSI.
        Если PSI для переменной выше порога - переменная макрируется
        0 (не использовать для дальнейшего анализа), если ниже
        порога - маркируется 1 (использовать для дальнейшего анализа).

    categorical_features: List[str], optional, default = None
        Список категориальных признаков для анализа.
        Опциональный параметр, по умолчанию, не используется, т.е.
        категориальные признаки отсутствуют.

    n_bins: int, optional, default = 20
        Количество бинов, на которые разбивается выборка.

    min_value: float, optional, default = 0.005
        Значение которое используется, если рассчитанный psi = 0.

    bin_type: string, optional, default = "quanitles"
        Способ разбиения на бины: "quantiles" or "bins".
        При выборе "quantiles" - выборка будет разбита на n_bins
        квантилей, при выборке "bins" - выборка будет разбита на
        n_bins бакетов с равным шагом между бакетами.
        Иные значения приводят к возникновению ValueError.

    Attributes
    ----------
    scores_: Dict[str, float]
        Словарь со значениями PSI,
        ключ словаря - название признака, значение - PSI-score.

    """
    def __init__(self,
                 threshold: float,
                 categorical_features: Optional[List[str]] = None,
                 bin_type: str = "quantiles",
                 min_value: float = 0.005,
                 n_bins: int = 20):

        self.threshold = threshold
        self.categorical_features = categorical_features
        self.min_value = min_value
        self.n_bins = n_bins
        if bin_type in ["quantiles", "bins"]:
            self.bin_type = bin_type
        else:
            raise ValueError(
                "Incorrect bin_type value. Expected 'quantiles' or 'bins', "
                f"but {bin_type} is transferred."
            )
        self.scores = {}

    def calculate_bins(self, data: pd.Series) -> np.array:
        """
        Вычисление границ бинов для разбиения выборки.

        Parameters
        ----------
        data: pandas.Series, shape = [n_samples, ]
            наблюдения из train-выборки.

        Returns
        -------
        bins: numpy.array, shape = [self.n_bins + 1]
            Список с границами бинов.

        """
        if self.bin_type == "quantiles":
            bins = np.linspace(0, 100, self.n_bins + 1)
            bins = [np.nanpercentile(data, x) for x in bins]

        else:
            bins = np.linspace(data.min(), data.max(), self.n_bins + 1)

        return np.unique(bins)

    def calculate_psi_in_bin(self, expected_score, actual_score) -> float:
        """
        Вычисление значения psi для одного бакета.

        Осуществляется проверка на равенство нулю expected_score и
        actual_score: если один из аргументов равен нулю, то его
        значение заменяется на self.min_value.

        Parameters
        ----------
        expected_score: float
            Ожидаемое значение.

        actual_score: float
            Наблюдаемое значение.

        Returns
        -------
        value: float
            Значение psi в бине.

        """
        if expected_score == 0:
            expected_score = self.min_value
        if actual_score == 0:
            actual_score = self.min_value

        value = (expected_score - actual_score)
        value = value * np.log(expected_score / actual_score)

        return value

    def calculate_psi(self, expected: pd.Series, actual: pd.Series, bins) -> float:
        """
        Расчет PSI для одной переменной.

        Parameters
        ----------
        expected: pandas.Series, shape = [n_samples_e, ]
            Наблюдения из train-выборки.

        actual: pandas.Series, shape = [n_samples_o, ]
            Наблюдения из test-выборки.

        bins: pandas.Series, shape = [self.n_bins, ]
            Бины для расчета PSI.

        Returns
        -------
        psi_score: float
            PSI-значение для данной пары выборок.

        """
        expected_score = np.histogram(expected.fillna(-9999), bins)[0]
        expected_score = expected_score / expected.shape[0]

        actual_score = np.histogram(actual.fillna(-9999), bins)[0]
        actual_score = actual_score / actual.shape[0]

        psi_score = np.sum(
            self.calculate_psi_in_bin(exp_score, act_score)
            for exp_score, act_score in zip(expected_score, actual_score)
        )

        return psi_score

    def calculate_numeric_psi(self, expected: pd.Series, actual: pd.Series) -> float:
        """
        Вычисление PSI для числовой переменной.

        Parameters
        ----------
        expected: pandas.Series, shape = [n_samples_e, ]
            Наблюдения из train-выборки.

        actual: pandas.Series, shape = [n_samples_o, ]
            Наблюдения из test-выборки.

        Returns
        -------
        psi_score: float
            PSI-значение для данной пары выборок.

        """
        bins = self.calculate_bins(expected)
        psi_score = self.calculate_psi(expected, actual, bins)
        return psi_score

    def calculate_categorical_psi(self, expected: pd.Series, actual: pd.Series) -> float:
        """
        Вычисление PSI для категориальной переменной.
        PSI рассчитывается для каждого уникального значения категории.

        Parameters
        ----------
        expected: pandas.Series, shape = [n_samples_e, ]
            Наблюдения из train-выборки.

        actual: pandas.Series, shape = [n_samples_o, ]
            Наблюдения из test-выборки.

        Returns
        -------
        psi_score: float
            PSI-значение для данной пары выборок.

        """
        bins = np.unique(expected).tolist()
        psi_score = self.calculate_psi(expected, actual, bins)
        return psi_score

    def fit(self, X, y=None):
        """
        Вычисление PSI-значения для всех признаков.

        Parameters
        ----------
        X: pandas.DataFrame, shape = [n_samples, n_features]
            Матрица признаков для обучения.

        y: pandas.DataFrame, shape = [n_samples, n_features]
            Матрица признаков для тестирования.

        Returns
        -------
        self
        """
        missed_columns = list(set(X.columns) - set(y.columns))

        if missed_columns:
            raise MissedColumnError(
                f"Missed {list(missed_columns)} columns in data.")

        if self.categorical_features:
            numeric_features = list(
                set(X.columns) - set(self.categorical_features)
            )
            for feature in self.categorical_features:
                self.scores[feature] = self.calculate_categorical_psi(
                    X[feature], y[feature]
                )
        else:
            numeric_features = X.columns

        for feature in tqdm(numeric_features):
            self.scores[feature] = self.calculate_numeric_psi(
                X[feature], y[feature]
            )
        return self

    def transform(self, X, y=None) -> pd.DataFrame:
        """
        Отбор переменных по self.threshold.
        Если PSI-score для переменной выше порога, то переменная
        помечается 0 (не использовать для дальнейшего анализа), если ниже
        порога - маркируется 1 (использовать для дальнейшего анализа).

        Parameters
        ----------
        data: pandas.DataFrame, shape = [n_samples, n_features]
            Матрица признаков для обучения.

        target: pandas.DataFrame, shape = [n_samples, n_features]
            Матрица признаков для тестирования.

        Returns
        -------
        scores: pandas.DataFrame, shape = [n_features, 3]
            Датафрейм с PSI-анализом переменных.

        """
        check_is_fitted(self, "scores")
        scores = pd.Series(self.scores)
        scores = pd.DataFrame({"Variable": scores.index, "PSI": scores.values})
        scores["Selected"] = np.where(scores.PSI < self.threshold, 1, 0)
        scores = scores.sort_values(by="PSI")

        mask = scores["Selected"] == 1
        self.used_features = scores.loc[mask, "Variable"].tolist()

        return scores.reset_index(drop=True)

In [ ]:
features_to_score = numerical_features.drop(["target"], axis=1).columns.tolist()
psi = PSI(threshold=0.01)

In [ ]:
score = psi.fit_transform(
    X=X_train[features_to_score],
    y=X_test[features_to_score]
)

In [ ]:
score['Selected'].unique()

Т.е все признаки мы можем использовать в обучении, т.к. они все прошли критерий PSI в 0.01

## Adversarial Validation

In [ ]:
X_adv = pd.concat([
    X_train[features_to_score], X_test[features_to_score]
])
y_adv = np.hstack(
    (np.ones(X_train.shape[0]), np.zeros(X_test.shape[0])))

X_adv = X_adv.reset_index(drop=True)

In [ ]:
%%time
parameters = {"max_depth": 4, "n_estimators": 50, "random_state": 27, "n_jobs": 2}
model = xgb.XGBClassifier(**parameters)

cv = cross_val_score(
    estimator=model,
    X=X_adv, y=y_adv,
    scoring="roc_auc",
    cv=KFold(n_splits=3, shuffle=True, random_state=27)
)

print(f"CV-score: {round(np.mean(cv), 4)}")

In [ ]:
Окончательно убедились, что трейн и тест из одного датасета и практически статистически неразличимы

In [ ]:
import gc
gc.collect()

In [ ]:
model.fit(X_adv, y_adv)

importance = pd.DataFrame({
    "feature": X_adv.columns,
    "adversarial_importance": model.feature_importances_
})
importance = importance.sort_values(by="adversarial_importance", ascending=False)
importance = importance.head(n=30)

plt.figure(figsize=(20, 5))
sns.barplot(x=importance["feature"], y=importance["adversarial_importance"], palette="viridis")
plt.title("Top Importance Features for Adversarial Validation (train vs test)", size=14)

In [ ]:
Все выводы по EDA:
    - убедились (несколькими способами), что трейн и тест из одного датасета и практически статистически неразличимы
    - сильный дисбаланс между бинарными классами в целевой переменной, надо делать балансировку
    - почти все признаки схожи с нормальными распределениями, но такими не являются. Имеют место скошенности в
    распределениях
    - в признаках нет пропусков, как и в целевой переменной.
    - все признаки - непрерывные, числовые, целевая переменная - бинарная
    - имеют место быть выбросы малой и средней величины (по сравнению с пиком) в пару десятков фич, явные их причины непонятны
    - все признаки мы можем использовать в обучении, т.к. они все прошли критерий PSI в 0.01
    - по результатм WOE/IV теста Согласно критерию 0.02, можно оставить 30 фич с показателем IV более 0.02,
    а именно: 139, 6, 174, 76, 146, 110, 12, 26, 21, 148, 34, 22, 115, 165, 80, 133, 170, 53, 166, 81, 149,
    99, 13, 190, 67, 169, 0, 78, 198, 192.
    До этого мы получили 20 наиболее (нелинейно) влиятельных фич: 81, 12, 139, 26, 53, 110, 174, 109, 22, 146,
    166, 80, 6, 165, 0, 78, 13, 179, 148, 108 Видим, что кроме 109, 179 и 108 признака, остальные совпадают. 
    Оставляем их. В самом начале мы отметили фичи: 2, 9, 12, 13, 16, 37, 40, 41, 53, 80, 81, 101, 108,
    126, 134, 153, 158, 175, 188, 196, из них можно оставить 108-ую
    - Итак, получаем 20 наиболее (нелинейно) влиятельных фич: 81, 12, 139, 26, 53, 110, 174, 109, 22, 146, 166,
    80, 6, 165, 0, 78, 13, 179, 148, 108.
    Ранее мы отметили следующие фичи: 2, 9, 12, 13, 16, 37, 40, 41, 53, 80, 81, 101, 108, 126, 134, 153, 158, 175,
    188, 196
    Видим, что 6 фич: 12, 13, 53, 80, 81, 108 сильно влияют на цель, при этом имеют выбросы, соответственно имеет
    смысл сделать на основе выбросов новые фичи - например, в выбросе - "1", в остальных местах "0"
    
    Т.о. можно оставить 18 фич: "var_0", "var_6", "var_12", "var_13", "var_22",
    "var_26", "var_53", "var_78", "var_80", "var_81", "var_108", "var_110",
    "var_139", "var_146", "var_148", "var_165", "var_166", "var_174" и на основе 6 фич: 12, 13, 53, 80, 81, 108
    сделать новые фичи